In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
import os
parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)

In [6]:
import json
path = '.'
file_name = 'java-test-(train_small).jsonl'
file_path = os.path.join(path, file_name)
data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            json_object = json.loads(line)
            data.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line.strip()} - {e}")

In [11]:
from models.llm import load_model, generate_comments
from prompts.templates import prepare_prompt

# Load model and tokenizer
model_name = "codellama/CodeLlama-7b-Instruct-hf"
model, tokenizer = load_model(model_name)
model = model.to("cuda")
tokenizer = tokenizer.to("cuda")

# Sample test input
test_input = {"code": data[0]['functions'][0]['code']}
few_shot_examples = data[0]['functions'][0]['CodeBERT']
for example in few_shot_examples:
    example['comment'] = example['docstring']

# Test different prompt strategies
prompt_types = ["zero-shot", "few-shot", "cot", "critique", "expert"]

for prompt_type in prompt_types:
    print(f"\nTesting {prompt_type} Prompt Strategy:")
    print('===========================')
    prompt = prepare_prompt(test_input, data_type="function", prompt_type=prompt_type, few_shot_examples=few_shot_examples)
    for p in prompt:
        print(f"{p['role']}: {p['content']}")
        print('=======================')
    comments = generate_comments(model, tokenizer, prompt, prompt_type=prompt_type)
    print(f"Generated Comments:\n{comments}")
    print('=======================')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 11.63 GiB of which 6.88 MiB is free. Including non-PyTorch memory, this process has 11.60 GiB memory in use. Of the allocated memory 11.30 GiB is allocated by PyTorch, and 85.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)